<a href="https://colab.research.google.com/github/Melissahachemi/classificateur/blob/main/Mod%C3%A8le_G%C3%A9n%C3%A9ratif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math

#recupération des données

!wget https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv
dataset = "IMDb_Reviews.csv"
df = pd.read_csv(dataset)

### Étape 1 : Mélanger le DataFrame

df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# - `sample(frac=1)` mélange le DataFrame en sélectionnant 100% des lignes au hasard.
# - `random_state` assure que le mélange est reproductible.
# - `reset_index(drop=True)` réinitialise l'index sans ajouter l'ancien index au DataFrame.

### Étape 2 : Diviser le DataFrame

# Calculer le point de division
split_idx = int(len(df_shuffled) * 0.8)

# Diviser en train et développement
df_train = df_shuffled[:split_idx]
df_dev = df_shuffled[split_idx:]

### Étape 3 : Sauvegarder les DataFrames dans des Fichiers

df_train.to_csv('imdb_train.csv', index=False)
df_dev.to_csv('imdb_dev.csv', index=False)

df_train_loaded = pd.read_csv("imdb_train.csv")
df_dev_loaded =  pd.read_csv("imdb_dev.csv")

## preparation du vocabulaire

class Vocabulaire:
    def __init__(self, token_unk='<unk>'):
        self.freq_mots = {}  # Pour compter la fréquence des mots
        self.mot_index = {token_unk: 0}  # Dictionnaire mot -> index, avec 'unk' comme premier mot
        self.index_mot = {0: token_unk}  # Dictionnaire index -> mot, avec 'unk' comme premier mot
        self.longueur = 1  # Nombre de mots uniques dans le vocabulaire, en commençant par 'unk'
        self.token_unk = token_unk

    def ajouter_texte(self, texte):
        for mot in texte.split():
            mot = mot.lower()  # Convertit le mot en minuscule
            if mot in self.freq_mots:
                self.freq_mots[mot] += 1
            else:
                self.freq_mots[mot] = 1

    def construire_vocabulaire(self, min_occurrence=15):
        # Commencez à 1 car 0 est réservé pour 'unk'
        index = 1
        for mot, freq in self.freq_mots.items():
            if freq >= min_occurrence and mot not in self.mot_index:
                self.mot_index[mot] = index
                self.index_mot[index] = mot
                index += 1
        # Mise à jour de la longueur après la construction
        self.longueur = len(self.mot_index)

    def transformer_texte_en_indices(self, texte):
        return [self.mot_index.get(mot.lower(), self.mot_index[self.token_unk]) for mot in texte.split()]

    def transformer_texte_en_vecteur(self, texte):
        """ Transformer le texte en vecteur de fréquences des mots du vocabulaire. """
        vecteur = np.zeros(self.longueur, dtype=int)
        for mot in texte.split():
            mot = mot.lower()
            index = self.mot_index.get(mot, self.mot_index[self.token_unk])
            vecteur[index] += 1
        return vecteur

    def construire_depuis_dataframe(self, df, colonne):
        """ Parcourt le DataFrame et ajoute tous les mots de la colonne spécifiée au vocabulaire. """
        df[colonne].str.lower().apply(self.ajouter_texte)
        self.construire_vocabulaire()
#calcule du nombre d'occurences de 0 et de 1  dans le train
nb_0=df_train_loaded['sentiment'].value_counts().get(0, 0)
nb_1=df_train_loaded['sentiment'].value_counts().get(1, 0)

#calcule de la taille de train
taille_train=len(df_train_loaded)

#calcule des proba
p_y_0= nb_0 / taille_train
p_y_1=nb_1 / taille_train

#creation du tableau des proba
tableau_probabilites = pd.DataFrame({
    'y': [0, 1],
    'P(y)': [p_y_0, p_y_1]
})

print(tableau_probabilites)


vocab=Vocabulaire()
vocab.construire_depuis_dataframe(df_train, 'review')

longueur_vocabulaire = vocab.longueur
print(f"La longueur du vocabulaire est : {longueur_vocabulaire}")

def calculer_probabilites_conditionnelles(df, vocab):
    """Calcule P(x/y) pour chaque mot x et chaque classe y."""

    # Initialisation des dictionnaires pour stocker les résultats
    probabilites_x_y_0 = {}
    probabilites_x_y_1 = {}

    # Taille du vocabulaire (pour le lissage de Laplace)
    taille_vocabulaire = vocab.longueur

    # Parcourir les données d'entraînement
    for index, row in df.iterrows():
        texte = row['review']
        y = row['sentiment']

        # Compter les occurrences de mots dans le texte pour chaque classe
        mots = texte.split()
        comptes_mots = {}
        for mot in mots:
            mot = mot.lower()  # Convertir en minuscules pour uniformité
            if mot in vocab.mot_index:  # Ignorer les mots inconnus
                if mot not in comptes_mots:
                    comptes_mots[mot] = 0
                comptes_mots[mot] += 1

        # Calculer les probabilités conditionnelles pour chaque mot
        for mot, count in comptes_mots.items():
            if y == 0:
                probabilites_x_y_0[mot] = (count + 1) / (sum(comptes_mots.values()) + taille_vocabulaire)
            else:
                probabilites_x_y_1[mot] = (count + 1) / (sum(comptes_mots.values()) + taille_vocabulaire)

    return probabilites_x_y_0, probabilites_x_y_1

# Calculer les probabilités conditionnelles
probabilites_x_y_0, probabilites_x_y_1 = calculer_probabilites_conditionnelles(df_train_loaded, vocab)

# Créer des DataFrames pour afficher les résultats
tableau_probabilites_x_y_0 = pd.DataFrame(list(probabilites_x_y_0.items()), columns=['mot', 'P(x/y=0)'])
tableau_probabilites_x_y_1 = pd.DataFrame(list(probabilites_x_y_1.items()), columns=['mot', 'P(x/y=1)'])

print("Tableau des probabilités P(x/y=0) :")
print(tableau_probabilites_x_y_0)

print("\nTableau des probabilités P(x/y=1) :")
print(tableau_probabilites_x_y_1)


## sac de mots
def creer_vecteur_sac_de_mots(vocab):
    """Crée un vecteur sac de mots initialisé avec des 0, avec la taille du vocabulaire."""
    vecteur = np.zeros(vocab.longueur, dtype=int)
    return vecteur

def remplir_vecteur_sac_de_mots(texte, vocab, vecteur):
    """Rempli le vecteur sac de mots avec les fréquences des mots du texte."""
    frequences_mots = {}  # Dictionnaire pour stocker les fréquences des mots dans le t
    for mot in texte.split():
        mot = mot.lower()
        index = vocab.mot_index.get(mot, 0)  # 0 pour le token <unk>
        vecteur[index] += 1
        if mot not in frequences_mots:
            frequences_mots[mot] = 0
        frequences_mots[mot] += 1
    return frequences_mots


# gerer l'underflow

def calculer_score(vecteur, probabilites_x_y, vocab):
    """Calcule le score pour une classe donnée."""
    score = 0
    for i, freq in enumerate(vecteur):
        if freq > 0:
            mot = vocab.index_mot.get(i)
            if mot in probabilites_x_y:
                proba = probabilites_x_y[mot]
                score += freq * math.log(proba)  # Utilisation du logarithme pour éviter les petits nombres
    return score

def predire_classe(texte, vocab, probabilites_x_y_0, probabilites_x_y_1):
    """Prédit la classe (0 ou 1) pour un texte donné."""
    vecteur = creer_vecteur_sac_de_mots(vocab)
    frequences_mots = remplir_vecteur_sac_de_mots(texte, vocab, vecteur)


    score_0 = calculer_score(vecteur, probabilites_x_y_0, vocab)
    score_1 = calculer_score(vecteur, probabilites_x_y_1, vocab)

    print("Vecteur sac de mots :", vecteur)
    print("Fréquences des mots dans le texte :", frequences_mots)
    print("Score pour la classe 0 :", score_0)
    print("Score pour la classe 1 :", score_1)

    if score_0 > score_1:
        return 0
    else:
        return 1


texte_dev = df_dev_loaded['review'].iloc[1]  # Prendre le premier texte du dev
classe_predite = predire_classe(texte_dev, vocab, probabilites_x_y_0, probabilites_x_y_1)

print(f"Texte du dev: {texte_dev}")
print(f"Classe prédite: {classe_predite}")

--2025-02-21 15:20:43--  https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65862309 (63M) [text/plain]
Saving to: ‘IMDb_Reviews.csv.1’

IMDb_Reviews.csv.1  100%[===================>]  62.81M   199MB/s    in 0.3s    

2025-02-21 15:20:43 (199 MB/s) - ‘IMDb_Reviews.csv.1’ saved [65862309/65862309]

   y    P(y)
0  0  0.4993
1  1  0.5007
La longueur du vocabulaire est : 27452
Tableau des probabilités P(x/y=0) :
             mot  P(x/y=0)
0              i  0.000072
1         wanted  0.000072
2             to  0.000145
3           love  0.000072
4           this  0.000109
...          ...       ...
27325   cuthbert  0.000212
27326  granger's  0.000073
27327     fatima  0.000